In [2]:
import pandas as pd
import numpy as np
import os

In [ ]:
os.chdir("c:/Users/Samsung/Documents/KPMG")
df = pd.read_csv('kpmg(keybert).csv',engine='python',encoding='utf-8',error_bad_lines=False)

### 전처리 과정

In [ ]:
#1.데이터프레임 -> 리스트

docs = []
for i in range(len(df)):
    list_ = df['kye_list'][i].replace("[", "").replace("]", "").replace("'", '')
    docs.append(list_)
docs

In [ ]:
#2. 토픽모델링을 위한 리스트 형태로 재설정

result = []
for i in range(len(docs)):
        str_list = docs[i]
        str_list = ''.join(s for s in str_list).replace(" ", "").split(',')
        result.append(str_list)
result

# LDA_토픽모델링 수행

출처: https://wikidocs.net/30708

In [6]:
#!pip install pyLDAvis 시각화패키지 설치

import gensim # LDA 모델 활용
import pyLDAvis.gensim_models # LDA 시각화용 패키지

from gensim import corpora # 단어 빈도수 계산 패키지
from collections import Counter # 단어 등장 횟수 카운트

In [7]:
# 단어 빈도수 계산 및 벡터화 

dictionary = corpora.Dictionary(result)
corpus = [dictionary.doc2bow(text) for text in result]
#print(corpus[1]) # 수행된 결과에서 두번째 값 확인

In [8]:
# LDA 학습 및 토픽 추출 

NUM_TOPICS = 5 # 토픽 수 
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=5)
topics = ldamodel.print_topics(num_words=10) #단어

In [9]:
# 토픽 추출 결과 확인

print(ldamodel.print_topics())

[(0, '0.113*"상각후원가측정금융자산" + 0.072*"공정가치측정금융자산" + 0.026*"측정불확실성" + 0.022*"경제환경" + 0.020*"법인세비용차감전순익" + 0.019*"누적환산차" + 0.017*"재고자산평가손실" + 0.017*"상각후원가측정항목" + 0.016*"추정불확실성" + 0.016*"연법인세자산"'), (1, '0.045*"연법인세자산" + 0.034*"채무상품금융자산" + 0.031*"일시적차" + 0.030*"경제환경" + 0.026*"순실현가능가치" + 0.024*"누적환산차" + 0.024*"전환상환우선주" + 0.023*"손상금융자산" + 0.022*"순공정가치" + 0.020*"충당부채거사건"'), (2, '0.043*"상각후원가측정금융자산" + 0.040*"경제환경" + 0.039*"당기손익인식금융부채" + 0.039*"공정가치측정금융자산" + 0.025*"경영환경" + 0.023*"법인세비용차감전순손익" + 0.021*"연법인세자산" + 0.021*"순공정가치" + 0.018*"재고자산평가손실" + 0.017*"중도상환특성"'), (3, '0.076*"가중평균유통보통주식수" + 0.027*"연법인세자산" + 0.022*"공정가치측정금융자산" + 0.022*"공정가치활성시장" + 0.022*"재고자산평가손실" + 0.021*"유형자산유형자산" + 0.021*"법인세비용차감전순익" + 0.018*"공정가치금융자산" + 0.017*"법인세비용차감전순손익" + 0.016*"경제환경"'), (4, '0.057*"채무상품금융자산" + 0.049*"손상금융자산" + 0.042*"경제환경" + 0.030*"충당부채거사건" + 0.026*"연법인세자산" + 0.025*"공정가치활성시장" + 0.021*"신용평가기관" + 0.020*"불확실성" + 0.019*"법인세비용차감전순익" + 0.018*"파생금융상품"')]


In [11]:
# LDA 시각화

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)
#pyLDAvis.save_html(vis, 'LDA_result.html') #HTML로 저장

c:\Users\Samsung\anaconda3\envs\jbnu\lib\site-packages\pyLDAvis\_prepare.py:244: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [17]:
# 기업 데이터 - LDA 결과 병합 

def make_topictable_per_doc(ldamodel, corpus):
    topic_table = pd.DataFrame()

    # 몇 번째 문서인지를 의미하는 문서 번호와 해당 문서의 토픽 비중을 한 줄씩 꺼내온다.
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list            
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
        # 각 문서에 대해서 비중이 높은 토픽순으로 토픽을 정렬한다.
        # EX) 정렬 전 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (10번 토픽, 5%), (12번 토픽, 21.5%), 
        # Ex) 정렬 후 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (12번 토픽, 21.5%), (10번 토픽, 5%)
        # 48 > 25 > 21 > 5 순으로 정렬이 된 것.

        # 모든 문서에 대해서 각각 아래를 수행
        for j, (topic_num, prop_topic) in enumerate(doc): #  몇 번 토픽인지와 비중을 나눠서 저장한다.
            if j == 0:  # 정렬을 한 상태이므로 가장 앞에 있는 것이 가장 비중이 높은 토픽
                topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic,4), topic_list]), ignore_index=True)
                # 가장 비중이 높은 토픽과, 가장 비중이 높은 토픽의 비중과, 전체 토픽의 비중을 저장한다.
            else:
                break
    return(topic_table)

In [ ]:
topictable = make_topictable_per_doc(ldamodel, corpus)
topictable = topictable.reset_index() # 문서 번호을 의미하는 열(column)로 사용하기 위해서 인덱스 열을 하나 더 만든다.
topictable.columns = ['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']
topictable

In [31]:
classify = topictable['가장 비중이 높은 토픽']

classify_df = pd.DataFrame({'classify':classify}) 
df.to_csv('classify_df.csv')